In [1]:
import pandas as pd
import numpy as np
import pywt
from hurst import compute_Hc
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import random

import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
torch.backends.cudnn.deterministic = True

In [31]:
torch.cuda.is_available()

False

In [32]:
df = pd.read_csv('../data/data.csv').drop('Unnamed: 0', axis=1)
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,5


## Data preprocessing and Wavelet feature extraction

In [33]:
class DataPreprocessing:
    
    @staticmethod
    def get_target(data):
        data['y'] = data['y'].apply(lambda t: int(t == 1))
        target = data['y']
        return data, target
    
    @staticmethod
    def get_balanced_indices(data, random_state):
        np.random.seed(random_state)
        shuffled_indices = np.random.permutation(data.index)
        X = data.loc[shuffled_indices]
        # getting first 6500 values
        return X.sort_values(by=data.columns[-1], ascending=False).iloc[:6500].index
    
    @staticmethod
    def plot_balanced_dataset(data, balanced_indices):
        plt.hist(data.iloc[balanced_indices][data.columns[-1]])
        
class WaveletFeatures:
    
    def __init__(self):
        pass

    def calculate_hurst(self, row):
        signal = row.iloc[:-1].values
        H, c, data = compute_Hc(signal, kind='change', simplified=True)
        return pd.Series({'hurst_exp': H, 'hurst_c': c})

    def get_hurst(self, data):
        data[['hurst_exp', 'hurst_c']] = data.apply(self.calculate_hurst, axis=1)
        return data
 
    def statistics_for_wavelet_transform(self, coefs):
        percentiles = [5, 25, 50, 75, 95]
        n = len(coefs)
        percentile_values = [np.nanpercentile(coefs, p) for p in percentiles]
        mean = np.nanmean(coefs)
        std = np.nanstd(coefs)
        var = np.nanvar(coefs)
        rms = np.sqrt(np.nanmean(coefs**2))
        return percentile_values + [mean, std, var, rms]

    def get_wavelet_features(self, data, target, kind="db4"):
        list_features = []
        for signal in range(len(data)):
            list_coeff = pywt.wavedec(data.iloc[signal], kind)
            features = []
            features.append(data.iloc[signal]["hurst_exp"])
            features.append(data.iloc[signal]["hurst_c"])
            for coeff in list_coeff:
                features += self.statistics_for_wavelet_transform(coeff)
            list_features.append(features)
        return self.create_df_wavelet(list_features, target)
    
    def create_df_wavelet(self, data, target):
        for i in range(len(data)):
            data[i].append(target[i])
        return pd.DataFrame(data)

Extracting features

In [47]:
df = pd.read_csv('../data/data.csv').drop('Unnamed: 0', axis=1)
df, target = DataPreprocessing.get_target(df)
balanced_indices = DataPreprocessing.get_balanced_indices(df, random_state=42)

wf = WaveletFeatures()
df = wf.get_hurst(df)
df = wf.get_wavelet_features(df, target,'db4')
df

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,0.546585,2.695749,-308.321013,-151.216527,7.404135,723.449162,776.917001,185.679314,420.960043,1.772074e+05,...,-15.854131,-5.558773,0.047096,5.772536,13.347925,-0.210035,9.121222,83.196688,9.123640,0
1,-0.254452,89.755181,-1243.000581,-488.612159,602.254479,1410.560020,1774.820173,506.884011,1217.918982,1.483327e+06,...,-40.671033,-6.574785,-0.634462,4.870545,42.662344,-0.683805,31.406413,986.362780,31.413856,1
2,0.742152,3.693757,-309.413814,-210.090841,-153.466544,-117.128242,-3.181903,-159.606594,95.538445,9.127594e+03,...,-7.354442,-3.472210,-0.256622,3.388624,6.927034,-0.249847,4.952315,24.525423,4.958613,0
3,0.767329,12.674929,-392.667105,-384.158663,-295.931344,-274.167229,-158.255685,-292.120739,107.005675,1.145021e+04,...,-5.060153,-2.330815,0.230629,2.307951,5.368949,0.038439,5.336453,28.477725,5.336591,0
4,0.647870,2.345903,-276.585954,-255.283369,-79.855665,12.241549,92.951925,-96.818803,139.465726,1.945069e+04,...,-9.754492,-3.206273,0.063130,3.667923,9.537925,0.177620,7.177783,51.520562,7.179980,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,0.360897,13.357499,-164.906640,-111.120502,-77.753449,86.759123,170.177351,-16.343038,123.865225,1.534259e+04,...,-3.343303,-1.618985,-0.353467,0.800135,3.391106,-0.190291,3.872117,14.993290,3.876790,0
11496,0.133973,13.041378,-284.907452,-91.298650,155.084114,211.667115,337.956886,76.708428,223.554768,4.997673e+04,...,-8.821328,-2.518248,-0.370076,1.272210,10.642185,-0.246161,15.690179,246.181715,15.692110,1
11497,0.580691,3.545585,-164.757338,3.012925,10.728446,33.163532,200.948947,22.386288,109.607442,1.201379e+04,...,-13.779522,-6.348407,0.249208,8.223007,17.413396,1.004669,10.056929,101.141814,10.106986,0
11498,0.606595,5.310635,-387.439152,-258.323654,-69.810319,-52.129498,83.868586,-125.896452,181.966551,3.311183e+04,...,-8.179212,-3.113531,0.286117,3.052020,8.023724,0.068804,6.252836,39.097954,6.253214,0


In [48]:
target[balanced_indices]

3509    1
1616    1
8121    1
629     1
7770    1
       ..
8710    0
388     0
836     0
9475    0
7410    0
Name: y, Length: 6500, dtype: int64

Dataset balancing and train-test split

In [49]:
X = df.iloc[balanced_indices].drop(columns=df.columns[-1])
y = target.iloc[balanced_indices] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
9474,0.024006,21.617115,-1387.122658,-1166.862826,-111.266798,799.402188,855.953449,-259.114806,893.949100,799144.994019,...,210.186074,-53.647930,-4.299047,-0.149752,6.933360,69.766631,1.849949,36.067801,1300.886279,36.115213
3358,0.648750,2.777617,-111.695791,-62.324083,-20.982287,68.275806,179.565999,10.148628,97.410428,9488.791398,...,17.362766,-8.525893,-3.483714,-0.341921,3.007473,7.353765,-0.243840,4.808387,23.120584,4.814566
8164,0.157829,11.024333,-778.621600,-561.961424,-391.202397,108.759170,521.472924,-209.420496,522.354053,272853.756614,...,98.445775,-32.887398,-5.261548,0.324391,10.551080,36.844848,1.620850,23.649546,559.301043,23.705025
5055,0.188378,18.805892,-499.375675,-121.521617,-95.988072,189.072966,513.444463,-9.750070,309.310426,95672.939615,...,41.658046,-10.708795,-2.690668,0.014231,2.661967,10.705159,-0.415933,7.508455,56.376890,7.519966
2356,0.491800,3.020966,-97.107882,-29.433954,-20.067830,26.465809,51.714414,-10.735908,48.323643,2335.174467,...,27.619949,-7.129860,-2.927981,-0.311683,2.653808,6.723491,-0.181498,7.693197,59.185287,7.695338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3882,0.095025,25.550078,-412.150740,-389.683649,-144.115111,55.920199,164.594101,-149.417062,241.048935,58104.589234,...,8.876586,-3.414377,-1.139200,0.107831,1.325302,3.679713,0.173828,5.393298,29.087659,5.396098
10734,0.435785,3.982007,-226.184021,-208.726821,-107.291092,79.259508,179.490846,-55.740978,148.380999,22016.920817,...,34.134295,-12.582697,-4.733684,-0.835060,6.915717,13.194750,-0.026150,8.469509,71.732584,8.469549
7264,0.581844,4.547961,-305.555472,-145.854651,-112.466517,-10.917598,20.314354,-99.594425,102.882555,10584.820180,...,6.695827,-3.050916,-1.447447,-0.219418,1.282156,3.896502,0.014900,2.817305,7.937209,2.817345
9075,0.518170,3.257182,-162.632701,-144.220819,-14.184791,66.721770,106.631412,-24.321202,104.546648,10930.001560,...,25.248227,-7.216084,-3.678869,0.100724,2.276838,7.351373,-0.220646,7.118455,50.672404,7.121874


## LSTM

In [51]:
class LSTMModel(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, l1, l2, lr):
        super().__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            batch_first=True) 

        self.dropout = nn.Dropout(p=0.1)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid() #Sigmoid
        
    def forward(self, x):
        output, (h_n, c_n) = self.lstm(x.to(device=self.device))
        output = self.dropout(output)
        output = self.fc(output[:, -1, :])
        output = self.sigmoid(output)
        return output.to(device=self.device)

In [54]:
class LSTMTrainer:
    
    def __init__(self, X_train, X_test, y_train, y_test, hidden_size, output_size, epochs, l1, l2, lr, patience):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
        self.X_train_LSTM = torch.from_numpy(X_train.values).float().reshape(X_train.shape[0], X_train.shape[1], 1).to(self.device)
        self.X_test_LSTM = torch.from_numpy(X_test.values).float().reshape(X_test.shape[0], X_test.shape[1], 1).to(self.device)
        self.y_train_LSTM = torch.from_numpy(y_train.values).float().to(self.device)
        self.y_test_LSTM = torch.from_numpy(y_test.values).float().to(self.device)
        
        # input_size = 1
        self.model = LSTMModel(input_size=np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],1).shape[2], 
                               hidden_size=hidden_size, output_size=output_size, l1=l1, l2=l2, lr=lr).to(self.device)
        
        self.criterion = nn.BCELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.epochs = epochs
        self.patience = patience
        
        self.train_losses = []
        self.test_losses = []
        self.train_accs = []
        self.test_accs = []
        
        # for early stopping
        self.best_loss = float('inf')
        self.counter = 0
        
    def train(self):
        for epoch in range(self.epochs):
            self.optimizer.zero_grad()
            output = self.model(self.X_train_LSTM)
            
            loss = self.criterion(output.squeeze(), self.y_train_LSTM)
            
            l1_lambda, l2_lambda = 0.0001, 0.0001 
            l1_reg, l2_reg = torch.tensor(0., device=self.device), torch.tensor(0., device=self.device)
            for param in self.model.parameters():
                l2_reg += torch.norm(param, p=2)
                l1_reg += torch.norm(param, p=1)
            loss += l2_lambda * l2_reg 
            loss += l1_lambda * l1_reg 
            
            loss.backward()
            self.optimizer.step()
            self.train_losses.append(loss.item())

            with torch.no_grad():
                test_output = self.model(self.X_test_LSTM)
                test_loss = self.criterion(test_output.squeeze(), self.y_test_LSTM)
                
                l1_reg, l2_reg = torch.tensor(0., device=self.device), torch.tensor(0., device=self.device)
                for param in self.model.parameters():
                    l2_reg += torch.norm(param, p=2)
                    l1_reg += torch.norm(param, p=1)
                test_loss += l2_lambda * l2_reg 
                test_loss += l1_lambda * l1_reg 
                self.test_losses.append(test_loss.item())

                train_acc = self._accuracy(output, self.y_train_LSTM)
                self.train_accs.append(train_acc)
                test_acc = self._accuracy(test_output, self.y_test_LSTM)
                self.test_accs.append(test_acc)

                # Early stopping
                if test_loss < self.best_loss:
                    self.best_loss = test_loss
                    self.counter = 0
                else:
                    self.counter += 1
                    if self.counter >= self.patience:
                        print("Early stopping at epoch", epoch+1)
                        return
              
            print('Epoch {}/{}, Train Loss: {:.4f}, Train Acc: {:.4f}, Test Loss: {:.4f}, Test Acc: {:.4f}'.
                  format(epoch+1, self.epochs, loss.item(), train_acc, test_loss.item(), test_acc))
          
    def _accuracy(self, output, target):
        rounded_output = torch.round(output).squeeze()
        target = target.squeeze()
        correct = (rounded_output == target).sum().float()
        accuracy = correct / target.size(0)
        return accuracy

In [55]:
trainer = LSTMTrainer(X_train, X_test, y_train, y_test, 
                      hidden_size=50, output_size=1, 
                      epochs=500, l1=0.0001, l2=0.01, lr=0.001, patience=100)
trainer.train()

Epoch 1/500, Train Loss: 0.7676, Train Acc: 0.6485, Test Loss: 0.7613, Test Acc: 0.6415
Epoch 2/500, Train Loss: 0.7601, Train Acc: 0.6501, Test Loss: 0.7546, Test Acc: 0.6415
Epoch 3/500, Train Loss: 0.7526, Train Acc: 0.6503, Test Loss: 0.7475, Test Acc: 0.6420
Epoch 4/500, Train Loss: 0.7458, Train Acc: 0.6510, Test Loss: 0.7412, Test Acc: 0.6410
Epoch 5/500, Train Loss: 0.7394, Train Acc: 0.6521, Test Loss: 0.7348, Test Acc: 0.6406
Epoch 6/500, Train Loss: 0.7332, Train Acc: 0.6517, Test Loss: 0.7283, Test Acc: 0.6424
Epoch 7/500, Train Loss: 0.7270, Train Acc: 0.6501, Test Loss: 0.7218, Test Acc: 0.6424
Epoch 8/500, Train Loss: 0.7204, Train Acc: 0.6507, Test Loss: 0.7162, Test Acc: 0.6424
Epoch 9/500, Train Loss: 0.7146, Train Acc: 0.6544, Test Loss: 0.7108, Test Acc: 0.6466
Epoch 10/500, Train Loss: 0.7085, Train Acc: 0.6572, Test Loss: 0.7045, Test Acc: 0.6531
Epoch 11/500, Train Loss: 0.7015, Train Acc: 0.6604, Test Loss: 0.6979, Test Acc: 0.6517
Epoch 12/500, Train Loss: 0.69

Epoch 94/500, Train Loss: 0.2451, Train Acc: 0.9343, Test Loss: 0.2325, Test Acc: 0.9436
Epoch 95/500, Train Loss: 0.2395, Train Acc: 0.9382, Test Loss: 0.2258, Test Acc: 0.9436
Epoch 96/500, Train Loss: 0.2337, Train Acc: 0.9398, Test Loss: 0.2260, Test Acc: 0.9445
Epoch 97/500, Train Loss: 0.2302, Train Acc: 0.9440, Test Loss: 0.2227, Test Acc: 0.9450
Epoch 98/500, Train Loss: 0.2276, Train Acc: 0.9412, Test Loss: 0.2270, Test Acc: 0.9455
Epoch 99/500, Train Loss: 0.2283, Train Acc: 0.9431, Test Loss: 0.2196, Test Acc: 0.9417
Epoch 100/500, Train Loss: 0.2288, Train Acc: 0.9392, Test Loss: 0.2218, Test Acc: 0.9464
Epoch 101/500, Train Loss: 0.2235, Train Acc: 0.9419, Test Loss: 0.2208, Test Acc: 0.9455
Epoch 102/500, Train Loss: 0.2237, Train Acc: 0.9428, Test Loss: 0.2128, Test Acc: 0.9455
Epoch 103/500, Train Loss: 0.2192, Train Acc: 0.9437, Test Loss: 0.2128, Test Acc: 0.9459
Epoch 104/500, Train Loss: 0.2169, Train Acc: 0.9444, Test Loss: 0.2124, Test Acc: 0.9478
Epoch 105/500, T

Epoch 186/500, Train Loss: 0.1579, Train Acc: 0.9600, Test Loss: 0.1459, Test Acc: 0.9683
Epoch 187/500, Train Loss: 0.1582, Train Acc: 0.9598, Test Loss: 0.1479, Test Acc: 0.9650
Epoch 188/500, Train Loss: 0.1571, Train Acc: 0.9582, Test Loss: 0.1468, Test Acc: 0.9683
Epoch 189/500, Train Loss: 0.1567, Train Acc: 0.9589, Test Loss: 0.1456, Test Acc: 0.9674
Epoch 190/500, Train Loss: 0.1558, Train Acc: 0.9603, Test Loss: 0.1443, Test Acc: 0.9669
Epoch 191/500, Train Loss: 0.1569, Train Acc: 0.9587, Test Loss: 0.1459, Test Acc: 0.9683
Epoch 192/500, Train Loss: 0.1553, Train Acc: 0.9594, Test Loss: 0.1447, Test Acc: 0.9664
Epoch 193/500, Train Loss: 0.1553, Train Acc: 0.9587, Test Loss: 0.1436, Test Acc: 0.9669
Epoch 194/500, Train Loss: 0.1546, Train Acc: 0.9600, Test Loss: 0.1440, Test Acc: 0.9683
Epoch 195/500, Train Loss: 0.1540, Train Acc: 0.9594, Test Loss: 0.1432, Test Acc: 0.9683
Epoch 196/500, Train Loss: 0.1546, Train Acc: 0.9582, Test Loss: 0.1428, Test Acc: 0.9664
Epoch 197/

Epoch 278/500, Train Loss: 0.1342, Train Acc: 0.9637, Test Loss: 0.1271, Test Acc: 0.9716
Epoch 279/500, Train Loss: 0.1339, Train Acc: 0.9639, Test Loss: 0.1283, Test Acc: 0.9697
Epoch 280/500, Train Loss: 0.1342, Train Acc: 0.9628, Test Loss: 0.1276, Test Acc: 0.9716
Epoch 281/500, Train Loss: 0.1339, Train Acc: 0.9649, Test Loss: 0.1271, Test Acc: 0.9725
Epoch 282/500, Train Loss: 0.1326, Train Acc: 0.9639, Test Loss: 0.1271, Test Acc: 0.9692
Epoch 283/500, Train Loss: 0.1330, Train Acc: 0.9637, Test Loss: 0.1264, Test Acc: 0.9711
Epoch 284/500, Train Loss: 0.1325, Train Acc: 0.9644, Test Loss: 0.1265, Test Acc: 0.9725
Epoch 285/500, Train Loss: 0.1326, Train Acc: 0.9637, Test Loss: 0.1265, Test Acc: 0.9725
Epoch 286/500, Train Loss: 0.1327, Train Acc: 0.9642, Test Loss: 0.1263, Test Acc: 0.9734
Epoch 287/500, Train Loss: 0.1322, Train Acc: 0.9635, Test Loss: 0.1277, Test Acc: 0.9711
Epoch 288/500, Train Loss: 0.1311, Train Acc: 0.9656, Test Loss: 0.1263, Test Acc: 0.9730
Epoch 289/

Epoch 370/500, Train Loss: 0.1209, Train Acc: 0.9679, Test Loss: 0.1201, Test Acc: 0.9753
Epoch 371/500, Train Loss: 0.1207, Train Acc: 0.9674, Test Loss: 0.1216, Test Acc: 0.9688
Epoch 372/500, Train Loss: 0.1242, Train Acc: 0.9649, Test Loss: 0.1208, Test Acc: 0.9744
Epoch 373/500, Train Loss: 0.1235, Train Acc: 0.9672, Test Loss: 0.1185, Test Acc: 0.9716
Epoch 374/500, Train Loss: 0.1213, Train Acc: 0.9660, Test Loss: 0.1175, Test Acc: 0.9734
Epoch 375/500, Train Loss: 0.1203, Train Acc: 0.9662, Test Loss: 0.1207, Test Acc: 0.9739
Epoch 376/500, Train Loss: 0.1215, Train Acc: 0.9683, Test Loss: 0.1196, Test Acc: 0.9692
Epoch 377/500, Train Loss: 0.1206, Train Acc: 0.9674, Test Loss: 0.1188, Test Acc: 0.9716
Epoch 378/500, Train Loss: 0.1204, Train Acc: 0.9679, Test Loss: 0.1208, Test Acc: 0.9734
Epoch 379/500, Train Loss: 0.1212, Train Acc: 0.9672, Test Loss: 0.1214, Test Acc: 0.9688
Epoch 380/500, Train Loss: 0.1233, Train Acc: 0.9665, Test Loss: 0.1176, Test Acc: 0.9748
Epoch 381/

Epoch 462/500, Train Loss: 0.1128, Train Acc: 0.9713, Test Loss: 0.1151, Test Acc: 0.9748
Epoch 463/500, Train Loss: 0.1149, Train Acc: 0.9692, Test Loss: 0.1121, Test Acc: 0.9758
Epoch 464/500, Train Loss: 0.1106, Train Acc: 0.9715, Test Loss: 0.1127, Test Acc: 0.9730
Epoch 465/500, Train Loss: 0.1124, Train Acc: 0.9704, Test Loss: 0.1130, Test Acc: 0.9776
Epoch 466/500, Train Loss: 0.1130, Train Acc: 0.9708, Test Loss: 0.1132, Test Acc: 0.9762
Epoch 467/500, Train Loss: 0.1105, Train Acc: 0.9715, Test Loss: 0.1132, Test Acc: 0.9734
Epoch 468/500, Train Loss: 0.1117, Train Acc: 0.9720, Test Loss: 0.1114, Test Acc: 0.9758
Epoch 469/500, Train Loss: 0.1106, Train Acc: 0.9708, Test Loss: 0.1122, Test Acc: 0.9748
Epoch 470/500, Train Loss: 0.1110, Train Acc: 0.9708, Test Loss: 0.1122, Test Acc: 0.9734
Epoch 471/500, Train Loss: 0.1116, Train Acc: 0.9713, Test Loss: 0.1114, Test Acc: 0.9758
Epoch 472/500, Train Loss: 0.1101, Train Acc: 0.9711, Test Loss: 0.1121, Test Acc: 0.9753
Epoch 473/

## GRU

In [56]:
class GRUModel(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, l1, l2, lr):
        super().__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.dropout = nn.Dropout(p=0.1)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        self.l1 = l1
        self.l2 = l2
        self.lr = lr
        
    def forward(self, x):
        output, h_n = self.gru(x.to(device=self.device))
        output = self.dropout(output)
        output = self.fc(output[:, -1, :])
        output = self.sigmoid(output)
        return output


In [57]:
class GRUTrainer:
    
    def __init__(self, X_train, X_test, y_train, y_test, hidden_size, output_size, epochs, l1, l2, lr, patience):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.X_train_GRU = torch.from_numpy(X_train.values).float().reshape(X_train.shape[0], X_train.shape[1], 1).to(self.device)
        self.X_test_GRU = torch.from_numpy(X_test.values).float().reshape(X_test.shape[0], X_test.shape[1], 1).to(self.device)
        self.y_train_GRU = torch.from_numpy(y_train.values).float().to(self.device)
        self.y_test_GRU = torch.from_numpy(y_test.values).float().to(self.device)

        self.model = GRUModel(input_size=np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],1).shape[2], 
                              hidden_size=hidden_size, output_size=output_size, l1=l1, l2=l2, lr=lr).to(self.device)
        self.criterion = nn.BCELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.epochs = epochs
        self.patience = patience

        self.train_losses = []
        self.test_losses = []
        self.train_accs = []
        self.test_accs = []

        self.best_loss = float('inf')
        self.counter = 0

    def train(self):
        for epoch in range(self.epochs):
            self.optimizer.zero_grad()
            output = self.model(self.X_train_GRU)
            loss = self.criterion(output.squeeze(), self.y_train_GRU)
            l2_lambda = 0.0001 
            l2_reg = torch.tensor(0., device=self.device)
            l1_lambda = 0.0001
            l1_reg = torch.tensor(0., device=self.device)
            for param in self.model.parameters():
                l2_reg += torch.norm(param, p=2).to(self.device)
                l1_reg += torch.norm(param, p=1).to(self.device)
            loss += l2_lambda * l2_reg 
            loss += l1_lambda * l1_reg 
            loss.backward()
            self.optimizer.step()
            self.train_losses.append(loss.item())

            with torch.no_grad():
                test_output = self.model(self.X_test_GRU)
                test_loss = self.criterion(test_output.squeeze(), self.y_test_GRU)
                l2_reg = torch.tensor(0., device=self.device)
                l1_reg = torch.tensor(0., device=self.device)
                for param in self.model.parameters():
                    l2_reg += torch.norm(param, p=2).to(self.device)
                    l1_reg += torch.norm(param, p=1).to(self.device)
                test_loss += l2_lambda * l2_reg 
                test_loss += l1_lambda * l1_reg 
                self.test_losses.append(test_loss.item())

                train_acc = self._accuracy(output, self.y_train_GRU)
                self.train_accs.append(train_acc)
                test_acc = self._accuracy(test_output, self.y_test_GRU)
                self.test_accs.append(test_acc)

                if test_loss < self.best_loss:
                    self.best_loss = test_loss
                    self.counter = 0
                else:
                    self.counter += 1
                    if self.counter >= self.patience:
                        print("Early stopping at epoch", epoch+1)
                        return

            print('Epoch {}/{}, Train Loss: {:.4f}, Train Acc: {:.4f}, Test Loss: {:.4f}, Test Acc: {:.4f}'.format(epoch+1, self.epochs, loss.item(), train_acc, test_loss.item(), test_acc))
          
    def _accuracy(self, output, target):
        rounded_output = torch.round(output).squeeze()
        target = target.squeeze()
        correct = (rounded_output == target).sum().float()
        accuracy = correct / target.size(0)
        return accuracy

In [60]:
trainer_gru = GRUTrainer(X_train, X_test, y_train, y_test, 
                      hidden_size=50, output_size=1, 
                      epochs=500, l1=0.0001, l2=0.01, lr=0.001, patience=10)
trainer_gru.train()  

Epoch 1/500, Train Loss: 0.8098, Train Acc: 0.3486, Test Loss: 0.7918, Test Acc: 0.3548
Epoch 2/500, Train Loss: 0.7953, Train Acc: 0.3483, Test Loss: 0.7770, Test Acc: 0.3594
Epoch 3/500, Train Loss: 0.7787, Train Acc: 0.3499, Test Loss: 0.7634, Test Acc: 0.3800
Epoch 4/500, Train Loss: 0.7644, Train Acc: 0.3720, Test Loss: 0.7519, Test Acc: 0.4252
Epoch 5/500, Train Loss: 0.7512, Train Acc: 0.4326, Test Loss: 0.7396, Test Acc: 0.5436
Epoch 6/500, Train Loss: 0.7390, Train Acc: 0.5474, Test Loss: 0.7274, Test Acc: 0.6723
Epoch 7/500, Train Loss: 0.7262, Train Acc: 0.6762, Test Loss: 0.7138, Test Acc: 0.7720
Epoch 8/500, Train Loss: 0.7166, Train Acc: 0.7552, Test Loss: 0.7043, Test Acc: 0.7944
Epoch 9/500, Train Loss: 0.7047, Train Acc: 0.7894, Test Loss: 0.6958, Test Acc: 0.8019
Epoch 10/500, Train Loss: 0.6945, Train Acc: 0.8030, Test Loss: 0.6856, Test Acc: 0.8037
Epoch 11/500, Train Loss: 0.6830, Train Acc: 0.8099, Test Loss: 0.6762, Test Acc: 0.7972
Epoch 12/500, Train Loss: 0.67

Epoch 94/500, Train Loss: 0.2040, Train Acc: 0.9433, Test Loss: 0.1855, Test Acc: 0.9552
Epoch 95/500, Train Loss: 0.2006, Train Acc: 0.9454, Test Loss: 0.1821, Test Acc: 0.9571
Epoch 96/500, Train Loss: 0.1991, Train Acc: 0.9467, Test Loss: 0.1867, Test Acc: 0.9543
Epoch 97/500, Train Loss: 0.2004, Train Acc: 0.9481, Test Loss: 0.1780, Test Acc: 0.9562
Epoch 98/500, Train Loss: 0.1984, Train Acc: 0.9454, Test Loss: 0.1749, Test Acc: 0.9557
Epoch 99/500, Train Loss: 0.1956, Train Acc: 0.9470, Test Loss: 0.1810, Test Acc: 0.9585
Epoch 100/500, Train Loss: 0.1964, Train Acc: 0.9502, Test Loss: 0.1725, Test Acc: 0.9590
Epoch 101/500, Train Loss: 0.1911, Train Acc: 0.9520, Test Loss: 0.1695, Test Acc: 0.9562
Epoch 102/500, Train Loss: 0.1934, Train Acc: 0.9449, Test Loss: 0.1708, Test Acc: 0.9604
Epoch 103/500, Train Loss: 0.1899, Train Acc: 0.9536, Test Loss: 0.1755, Test Acc: 0.9608
Epoch 104/500, Train Loss: 0.1910, Train Acc: 0.9541, Test Loss: 0.1686, Test Acc: 0.9590
Epoch 105/500, T

Epoch 186/500, Train Loss: 0.1527, Train Acc: 0.9617, Test Loss: 0.1349, Test Acc: 0.9748
Epoch 187/500, Train Loss: 0.1520, Train Acc: 0.9621, Test Loss: 0.1355, Test Acc: 0.9767
Epoch 188/500, Train Loss: 0.1520, Train Acc: 0.9617, Test Loss: 0.1349, Test Acc: 0.9748
Epoch 189/500, Train Loss: 0.1514, Train Acc: 0.9617, Test Loss: 0.1356, Test Acc: 0.9758
Epoch 190/500, Train Loss: 0.1511, Train Acc: 0.9621, Test Loss: 0.1347, Test Acc: 0.9758
Epoch 191/500, Train Loss: 0.1507, Train Acc: 0.9619, Test Loss: 0.1347, Test Acc: 0.9753
Epoch 192/500, Train Loss: 0.1505, Train Acc: 0.9621, Test Loss: 0.1331, Test Acc: 0.9762
Epoch 193/500, Train Loss: 0.1513, Train Acc: 0.9621, Test Loss: 0.1342, Test Acc: 0.9762
Epoch 194/500, Train Loss: 0.1506, Train Acc: 0.9621, Test Loss: 0.1343, Test Acc: 0.9758
Epoch 195/500, Train Loss: 0.1510, Train Acc: 0.9621, Test Loss: 0.1333, Test Acc: 0.9767
Epoch 196/500, Train Loss: 0.1499, Train Acc: 0.9617, Test Loss: 0.1334, Test Acc: 0.9753
Epoch 197/